In [26]:
#Testing looping through files and normalizing each one, exporting as csv
#eventually work to merge these .csvs??

import pandas as pd #df handling
import fastparquet #make sure that this is installed in your anaconda environment
import numpy as np #w pandas
import statistics as stat
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.datasets import make_classification
from feature_engine.selection import DropCorrelatedFeatures
import os
import json

#get the current directory
#Change this to match where you have your parquet files downloaded:
ids='TESTID_13Oct23'
directory_in_str='C:/Users/Sepro/Capstone/Inputs/'
#directory = os.fsencode(directory_in_str)

cwd = os.getcwd()
input_dir = os.path.join(cwd, 'Inputs') #this is the folder that contains a csv of all the files you're trying to process
#df_total = pd.read_parquet(os.path.join(input_dir, 'metadata_merged.csv'),
 #                               usecols=['Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_InChIKey'],
  #                              dtype={'Metadata_Plate': str})
filenames=pd.read_csv(os.path.join(input_dir,'Test_Parqs.csv'))
plates = filenames['FileName'].unique() #csv should have a header that says FileName with parquet names under it


plate_dfs = []
failed_plates = []
rows_removed = 0
total_rows = 0
    
for i, plate in enumerate(plates):
    #print(plate), prints are used for debugging
    file= directory_in_str+ plate + '.parquet' #concatonate parquet name w the appropriate directory so it can find ti
    df=pd.read_parquet(file, engine='fastparquet')
    
    #print(df.shape)
    
    #Separate out the metadata columns 
    metadata_cols = df.loc[:,df.columns.str.contains('Metadata')]
    
    #pulled straight from ryan, removing columns
        
    all_cols = df.columns
    to_exclude = ['Metadata', 'Image', 'BoundingBox', 'Center', 'EulerNumber', 'Count', 'Neighbors', 'FormFactor',
                  'Location', 'Overflow', 'mito_tubeness', 'Manders', 'MinIntensity',
                  'Cytoplasm_Texture_Contrast_DNA_10', 'Cytoplasm_Texture_DifferenceEntropy_DNA_10',
                  'Cytoplasm_Texture_Entropy_DNA_10', 'Cytoplasm_Texture_InfoMeas1_DNA_10',
                  'Cytoplasm_Texture_InfoMeas2_DNA_10', 'Cytoplasm_Texture_SumEntropy_DNA_1',
                  'Cytoplasm_Texture_SumVariance_DNA_10', 'Cytoplasm_Texture_Variance_DNA_10', 'CellsIncludingEdges']
    excluded_cols = np.full(len(all_cols), False)
    for cols in to_exclude:
        excluded_cols = excluded_cols | all_cols.str.contains(cols)
    df = df.loc[:, ~excluded_cols]
    #print(df.shape)
    
    with open(os.path.join(input_dir, 'columns_to_drop.json')) as f:
        correlated_features = json.load(f)
    df = df.drop(columns=correlated_features)
    
    #print(df.shape)

        

    data_numerical=df._get_numeric_data() #removing columns that have non quantitiative data

    df_normalized = (0.6745 * (data_numerical - data_numerical.median(axis=0))) / stats.median_abs_deviation(data_numerical, axis=0)


    df_normalized = df_normalized.dropna(axis=1) #this is what the problem was, remove inf and NaN data w this line

    
    #print(df_normalized.shape)
    
    #remove outliers
    n_MAD = 500
    threshold = 0.6745 * n_MAD
    rows_to_keep = (np.abs(df_normalized) < threshold).all(axis=1)
    df_normalized = df_normalized[rows_to_keep]
    metadata_cols = metadata_cols[rows_to_keep]
    
    #print(df_normalized.shape)
    
    dfs_full=pd.concat([metadata_cols, df_normalized, ], axis=1)
    
    #print(dfs_full.shape)

    #dropped_correlated = corrs.fit_transform(df_normalized)

    plate_dfs.append(dfs_full)
    
#exit for loop

# Aggregate and save plates
df_normalized_plates = pd.concat(plate_dfs)
output_dir = os.path.join(cwd, 'output')
outfile = os.path.join(output_dir, 'normalized_samples_' + str(n_MAD) + 'MAD_'+ids+'.csv')
df_normalized_plates.to_csv(outfile, index=False)
    
    
    

1053600674
(383, 964)
(366, 964)
(366, 967)
UL001647
(1472, 964)
(1470, 964)
(1470, 967)


In [25]:
df_normalized_plates.shape
    
    


(1836, 967)